In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-05-24T13:56:54.571010-04:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
from dask import dataframe as dd
from dask.delayed import delayed
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import optuna
import gc
import logging

In [4]:
%watermark --iversions

optuna : 2.10.1
logging: 0.5.1.2
dask   : 2022.1.0
xgboost: 1.6.2
pandas : 1.3.5
numpy  : 1.19.2



In [5]:
!nvidia-smi -L

GPU 0: Tesla V100-DGXS-32GB (UUID: GPU-d752af09-1f62-bf3b-4f70-78b84e9e41f6)
GPU 1: Tesla V100-DGXS-32GB (UUID: GPU-054a4a35-f98a-3ebc-9100-0f697c246b43)
GPU 2: Tesla V100-DGXS-32GB (UUID: GPU-454525d4-bebd-7fb8-0ba3-3b85e2f99abd)
GPU 3: Tesla V100-DGXS-32GB (UUID: GPU-af0fb74e-f5eb-0833-17ff-494cc6bdcee1)


In [6]:
cluster = LocalCUDACluster(n_workers=4)
client = Client(cluster)

In [7]:
!nvidia-smi

Wed May 24 13:57:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.142.00   Driver Version: 450.142.00   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0  On |                    0 |
| N/A   41C    P0    51W / 300W |    653MiB / 32499MiB |     13%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [8]:
train_folds = []
val_folds = []
train_ys = []
val_ys = []

for i in range(5):
    print(f'Loading fold {i}')
    train_fold_d = delayed(pd.read_csv)(f'../input/xgtrain_fold_{i}.csv.gz')
    train_fold = dd.from_delayed(train_fold_d)
    
    val_fold_d = delayed(pd.read_csv)(f'../input/xgval_fold_{i}.csv.gz')
    val_fold = dd.from_delayed(val_fold_d)
    
    
    train_y = train_fold['target']
    train_fold = train_fold[train_fold.columns.difference(['target'])]
    
    val_y = val_fold['target']
    val_fold = val_fold[val_fold.columns.difference(['target'])]
    
    train_folds.append(train_fold)
    val_folds.append(val_fold)
    
    train_ys.append(train_y)
    val_ys.append(val_y)

Loading fold 0
Loading fold 1
Loading fold 2
Loading fold 3
Loading fold 4


In [9]:
train = pd.read_csv('../input/train.csv.zip')

target = train['loss'].values
target

array([2213.18, 1283.6 , 3005.09, ..., 5762.64, 1562.87, 4751.72])

In [13]:
train_oof = np.zeros((target.shape[0],))

num_round = 1000

def objective(trial):
        
    params = {
        'objective': 'reg:squarederror', 
        'tree_method':'gpu_hist',  # 'gpu_hist','hist'
        'lambda': trial.suggest_loguniform('lambda',1e-3,10.0),
        'alpha': trial.suggest_loguniform('alpha',1e-3,10.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.3,1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.001,0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 1,300),
        'eval_metric': trial.suggest_categorical('eval_metric',['rmse']),

    }

    kf = KFold(5, shuffle=True, random_state=137)

    for i, (train_index, val_index) in enumerate(kf.split(train,target)):
        dtrain = xgb.dask.DaskDMatrix(client, train_folds[i].values, train_ys[i], enable_categorical=True)
        dval = xgb.dask.DaskDMatrix(client, val_folds[i].values, val_ys[i], enable_categorical=True)
        
        output = xgb.dask.train(client, params, dtrain, num_round)
        booster = output['booster']  # booster is the trained model
        booster.set_param({'predictor': 'gpu_predictor'})
        predictions = xgb.dask.predict(client, booster, dval)
        predictions = predictions.compute()
        train_oof[val_index] = predictions
        del dtrain, dval, output
        gc.collect()
        gc.collect()

    mae = mean_absolute_error(target, train_oof)
    
    return mae

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)  # Setup the root logger.
logger.addHandler(logging.FileHandler("optuna_xgb_output_0.log", mode="w"))

optuna.logging.enable_propagation()  # Propagate logs to the root logger.
optuna.logging.disable_default_handler()  # Stop showing logs in sys.stderr.

study = optuna.create_study(storage="sqlite:///xgb_optuna_allstate_0.db", study_name="five_fold_optuna_xgb_0", direction='minimize')

In [14]:
%%time
logger.info("Start optimization.")
study.optimize(objective, n_trials=3)

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 44% CPU time recently (threshold: 10%)
/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 74% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garb

CPU times: user 23.9 s, sys: 4.08 s, total: 28 s
Wall time: 3min 38s


In [15]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,NaN,7.474116,0.435754,rmse,0.001568,0.063664,15,221,0.572279,FAIL
1,1,1173.931790,1.151202,0.330074,rmse,0.873551,0.029280,12,297,0.802492,COMPLETE
2,2,1221.577226,0.027813,0.587490,rmse,0.002588,0.018707,3,57,0.708085,COMPLETE
3,3,1176.732863,5.312879,0.583720,rmse,0.262292,0.029229,19,236,0.800300,COMPLETE


In [16]:
%%time
study.optimize(objective, n_trials=5)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 75% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 76% CPU time recently (thre

CPU times: user 2min 42s, sys: 41.5 s, total: 3min 23s
Wall time: 36min 14s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,NaN,7.474116,0.435754,rmse,0.001568,0.063664,15,221,0.572279,FAIL
1,1,1173.931790,1.151202,0.330074,rmse,0.873551,0.029280,12,297,0.802492,COMPLETE
2,2,1221.577226,0.027813,0.587490,rmse,0.002588,0.018707,3,57,0.708085,COMPLETE
3,3,1176.732863,5.312879,0.583720,rmse,0.262292,0.029229,19,236,0.800300,COMPLETE
4,4,1179.044801,1.336278,0.543375,rmse,7.162421,0.025564,8,232,0.631366,COMPLETE


In [17]:
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.head()

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,NaN,7.474116,0.435754,rmse,0.001568,0.063664,15,221,0.572279,FAIL
1,1,1173.931790,1.151202,0.330074,rmse,0.873551,0.029280,12,297,0.802492,COMPLETE
2,2,1221.577226,0.027813,0.587490,rmse,0.002588,0.018707,3,57,0.708085,COMPLETE
3,3,1176.732863,5.312879,0.583720,rmse,0.262292,0.029229,19,236,0.800300,COMPLETE
4,4,1179.044801,1.336278,0.543375,rmse,7.162421,0.025564,8,232,0.631366,COMPLETE


In [18]:
%%time
study.optimize(objective, n_trials=100)
df = study.trials_dataframe(attrs=('number', 'value', 'params', 'state'))
df.to_csv('optuna_xgb_output_0.csv', index=False)
df.head()

/home/tunguz/anaconda3/lib/python3.7/site-packages/xgboost/dask.py:884: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
distributed.utils_perf - WARNING - full garbage collections took 82% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 82% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 82% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 82% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 83% CPU time recently (thre

CPU times: user 1h 10min 31s, sys: 15min 25s, total: 1h 25min 56s
Wall time: 13h 50min 17s


,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,NaN,7.474116,0.435754,rmse,0.001568,0.063664,15,221,0.572279,FAIL
1,1,1173.931790,1.151202,0.330074,rmse,0.873551,0.029280,12,297,0.802492,COMPLETE
2,2,1221.577226,0.027813,0.587490,rmse,0.002588,0.018707,3,57,0.708085,COMPLETE
3,3,1176.732863,5.312879,0.583720,rmse,0.262292,0.029229,19,236,0.800300,COMPLETE
4,4,1179.044801,1.336278,0.543375,rmse,7.162421,0.025564,8,232,0.631366,COMPLETE


In [19]:
df

,number,value,params_alpha,params_colsample_bytree,params_eval_metric,params_lambda,params_learning_rate,params_max_depth,params_min_child_weight,params_subsample,state
0,0,NaN,7.474116,0.435754,rmse,0.001568,0.063664,15,221,0.572279,FAIL
1,1,1173.931790,1.151202,0.330074,rmse,0.873551,0.029280,12,297,0.802492,COMPLETE
2,2,1221.577226,0.027813,0.587490,rmse,0.002588,0.018707,3,57,0.708085,COMPLETE
3,3,1176.732863,5.312879,0.583720,rmse,0.262292,0.029229,19,236,0.800300,COMPLETE
4,4,1179.044801,1.336278,0.543375,rmse,7.162421,0.025564,8,232,0.631366,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...
104,104,1170.045945,0.007738,0.423866,rmse,0.268928,0.009876,19,126,0.944663,COMPLETE
105,105,1169.989915,0.007728,0.452506,rmse,0.258777,0.010661,19,109,0.952994,COMPLETE
106,106,1170.537693,0.008312,0.458450,rmse,0.271196,0.009366,19,133,0.946633,COMPLETE
107,107,1170.879727,0.006603,0.422836,rmse,0.151230,0.010239,19,157,0.967572,COMPLETE
